### Producer 

The producer should connect to reddit via the credentials file 'creds.sh' and recieve the defined post data from a given subreddit.

In [1]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install praw

Note: you may need to restart the kernel to use updated packages.


Credentials

In [3]:
import os
import dotenv
env_file = 'creds.sh'
dotenv.load_dotenv(env_file, override=True)

CLIENT_ID = os.environ['CLIENT_ID']
SECRET_TOKEN = os.environ['SECRET_TOKEN']
USER_AGENT = 'MyBot/0.0.2'

In [4]:
host = "127.0.0.1"
port = 9998

Python Reddit API Wrapper

In [5]:
import praw

reddit = praw.Reddit(client_id=CLIENT_ID,
                     client_secret=SECRET_TOKEN,
                     user_agent=USER_AGENT)

Socket & Producer

In [6]:
import socket
import json
import praw

subred_name = "AskReddit+ask+AskHistorians+AskWomen+AskMen+AskCulinary+askphilosophy"

# Socket Prep
s = socket.socket()
s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
s.bind((host, port))
print("Listening on port: %s" % str(port))
s.listen() # listen(5)


def stream_json(subreddit):
    for comment in subreddit.stream.comments():
        try:
            post = comment.submission
            parent_id = str(comment.parent())
            prev_comment = reddit.comment(parent_id)

            prev_body = prev_comment.body
            comment_body = comment.body
            
            my_object = {
                "comment": comment_body,
                "prev_comment": prev_body,
                "post": post.selftext,
                "author": str(comment.author),
                "link_url": comment.link_url,
                "link_permalink": comment.link_permalink,
                "created_utc": comment.created_utc,
                "ups": comment.ups,
                "likes": comment.likes,
            }
            json_encoded = json.dumps(my_object)
            print(json_encoded)

            # Send through socket
            c, addr = s.accept()
            c.send(json_encoded.encode('utf-8'))
            c.close()
        except praw.exceptions.PRAWException as ex:
            pass

reddit = praw.Reddit(client_id=CLIENT_ID,
                     client_secret=SECRET_TOKEN,
                     user_agent=USER_AGENT)
subreddit = reddit.subreddit(subred_name)

stream_json(subreddit)

Listening on port: 9998
{"comment": "It's an egg that's been preserved in a mixture of clay, salt, quicklime, ash, and rice hulls. This makes the egg alkaline as fuck so nothing grows in it. Normally it takes like 7 weeks to months to make one. They're called century egg or thousand year egg but I don't think anyone has actually tried to create a literal century egg.", "prev_comment": "Tf is century egg, I mean I guess it\u2019s a hundred year old but how does that work. Dies one not die when he consumes it? How is it prepared?", "post": "", "author": "Noe_b0dy", "link_url": "https://www.reddit.com/r/AskReddit/comments/1dd8rkq/whats_the_weirdest_thing_youve_ever_eaten_and_how/", "link_permalink": "https://www.reddit.com/r/AskReddit/comments/1dd8rkq/whats_the_weirdest_thing_youve_ever_eaten_and_how/", "created_utc": 1718098109.0, "ups": 1, "likes": null}


Test Code

In [ ]:
from datetime import datetime
import json
import socket

# Method to return each post data from a particular subreddit
def stream_data(subreddit):
    for submission in subreddit.stream.submissions():
        post = {
            'title': submission.title,
            'date': str(datetime.utcfromtimestamp(submission.created_utc)),
            'author': str(submission.author),
            'subreddit': str(submission.subreddit)
        }
        yield post

host = "127.0.0.1"
port = 9998

# Defined socket
s = socket.socket()
s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
s.bind((host, port))
s.listen()
print("Here I'm waiting...")

# Define the subreddit
subred_name = "AskReddit"

# Producer
while True:
    c, addr = s.accept()
    print(f"Here I got a connection from {addr}")
    subreddit = reddit.subreddit(subred_name)
    
    for post in stream_data(subreddit):
        data_to_send = json.dumps(post)
        c.send(data_to_send.encode("utf-8"))
        print("message sent!")
        print(data_to_send)
        c.close()
        break

In [ ]:
from datetime import datetime
import json
import socket
import praw
import socket
import json
import threading

host = "127.0.0.1"
port = 9998
subred_name = "python+generativeAI+ArtificialInteligence+ChatGPTCoding"  


# Setup socket
s = socket.socket()
s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
s.bind((host, port))
print("Listening on port:", port)
s.listen(1)  # Listen for incoming connections

def stream_json(subreddit):
    client, addr = s.accept()
    print("Got connection from", addr)
    for comment in subreddit.stream.comments(skip_existing=True):
        try:
            post = comment.submission
            
            my_object = {
                "comment": comment.body,
                "post": post.selftext,
                "author": str(comment.author),
                "created_utc": comment.created_utc,
            }
            json_encoded = json.dumps(my_object)
            print(json_encoded)

            # Send through socket
            client.send(json_encoded.encode('utf-8'))
        except Exception as ex:
            print("Exception:", ex)
            continue

subreddit = reddit.subreddit(subred_name)
threading.Thread(target=stream_json, args=(subreddit,)).start()

In [ ]:
# Setup socket
s = socket.socket()
s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
s.bind((host, port))
s.listen(1)
print("Listening on port:", port)

def stream_data(subreddit):
    for submission in subreddit.stream.submissions(skip_existing=True):
        post = {
            'type': 'submission',
            'title': submission.title,
            'text': submission.selftext,
            'date': str(datetime.utcfromtimestamp(submission.created_utc)),
            'author': str(submission.author),
            'subreddit': str(submission.subreddit)
        }
        yield post

    for comment in subreddit.stream.comments(skip_existing=True):
        post = {
            'type': 'comment',
            'text': comment.body,
            'date': str(datetime.utcfromtimestamp(comment.created_utc)),
            'author': str(comment.author),
            'subreddit': str(comment.subreddit)
        }
        yield post

def handle_client(client_socket, subreddit):
    for post in stream_data(subreddit):
        data_to_send = json.dumps(post)
        client_socket.send(data_to_send.encode("utf-8"))
        print("Message sent!")
        print(data_to_send)

    client_socket.close()

def start_server():
    subreddit = reddit.subreddit(subred_name)
    while True:
        client_socket, addr = s.accept()
        print(f"Got connection from {addr}")
        client_handler = threading.Thread(target=handle_client, args=(client_socket, subreddit))
        client_handler.start()

start_server()